In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [3]:
torch.cuda.is_available()

True

In [4]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch(loss_log):
    model.train()
    for x, y in train_loader:
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss_log += list(loss.flatten().cpu().detach().numpy())
        loss.backward()
        optimizer.step()

In [5]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [6]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [7]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch")
loss_fn = nn.NLLLoss()

In [8]:
log_file = open("SGD_momentum.txt", "w")

## Epoch

In [9]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 1.7480
Validation accuracy: 0.7748, validation loss: 1.3486
Epoch 2
Training loss: 1.1531
Validation accuracy: 0.8178, validation loss: 1.0026
Epoch 3
Training loss: 0.9133
Validation accuracy: 0.8344, validation loss: 0.8397
Epoch 4
Training loss: 0.7876
Validation accuracy: 0.8442, validation loss: 0.7446
Epoch 5
Training loss: 0.7096
Validation accuracy: 0.8507, validation loss: 0.6821
Epoch 6
Training loss: 0.6559
Validation accuracy: 0.8560, validation loss: 0.6374
Epoch 7
Training loss: 0.6165
Validation accuracy: 0.8603, validation loss: 0.6038
Epoch 8
Training loss: 0.5860
Validation accuracy: 0.8647, validation loss: 0.5773
Epoch 9
Training loss: 0.5617
Validation accuracy: 0.8665, validation loss: 0.5557
Epoch 10
Training loss: 0.5417
Validation accuracy: 0.8704, validation loss: 0.5379
Epoch 11
Training loss: 0.5249
Validation accuracy: 0.8727, validation loss: 0.5229
Epoch 12
Training loss: 0.5106
Validation accuracy: 0.8748, validation loss: 0.5099
E

In [10]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8933, device='cuda:0'), tensor(0.4001, device='cuda:0'))
Valid: (tensor(0.8902, device='cuda:0'), tensor(0.4103, device='cuda:0'))


In [11]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.accelerate()

In [12]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [13]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9022, device='cuda:0'), tensor(0.3478, device='cuda:0'))
Valid: (tensor(0.8995, device='cuda:0'), tensor(0.3621, device='cuda:0'))


In [14]:
optimizer.param_groups[0]["method"] = "RRE"
optimizer.accelerate()

In [15]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [16]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9022, device='cuda:0'), tensor(0.3478, device='cuda:0'))
Valid: (tensor(0.8995, device='cuda:0'), tensor(0.3621, device='cuda:0'))


## Epoch average

In [27]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [28]:
log_file = open("SGD_momentum-avg.txt", "w")
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch_avg")

In [29]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 1.7550
Validation accuracy: 0.7809, validation loss: 1.3521
Epoch 2
Training loss: 1.1554
Validation accuracy: 0.8200, validation loss: 1.0038
Epoch 3
Training loss: 0.9144
Validation accuracy: 0.8337, validation loss: 0.8402
Epoch 4
Training loss: 0.7886
Validation accuracy: 0.8431, validation loss: 0.7456
Epoch 5
Training loss: 0.7107
Validation accuracy: 0.8499, validation loss: 0.6832
Epoch 6
Training loss: 0.6572
Validation accuracy: 0.8545, validation loss: 0.6389
Epoch 7
Training loss: 0.6179
Validation accuracy: 0.8582, validation loss: 0.6053
Epoch 8
Training loss: 0.5875
Validation accuracy: 0.8613, validation loss: 0.5788
Epoch 9
Training loss: 0.5632
Validation accuracy: 0.8658, validation loss: 0.5574
Epoch 10
Training loss: 0.5432
Validation accuracy: 0.8683, validation loss: 0.5395
Epoch 11
Training loss: 0.5263
Validation accuracy: 0.8697, validation loss: 0.5245
Epoch 12
Training loss: 0.5121
Validation accuracy: 0.8721, validation loss: 0.5115
E

In [30]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8925, device='cuda:0'), tensor(0.4011, device='cuda:0'))
Valid: (tensor(0.8888, device='cuda:0'), tensor(0.4114, device='cuda:0'))


In [31]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.accelerate()

In [32]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [33]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8963, device='cuda:0'), tensor(0.3821, device='cuda:0'))
Valid: (tensor(0.8913, device='cuda:0'), tensor(0.3936, device='cuda:0'))


In [34]:
optimizer.param_groups[0]["method"] = "RRE"
optimizer.accelerate()

In [35]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [36]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8964, device='cuda:0'), tensor(0.3819, device='cuda:0'))
Valid: (tensor(0.8915, device='cuda:0'), tensor(0.3934, device='cuda:0'))
